In [ ]:


# -*- coding: utf-8 -*-
"""
Created on Fri Jun  4 08:38:19 2021

 

@author: ashish_kumar2
"""

 

# Import Parame
import requests
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
import time

 


# Test Environment Credentials (v1.0)
t_current_version="1.0"
t_app_id="4fdff778-8d0f-48bd-be01-85926d58bcc4"
t_authoringKey = 'dfac75a1fd1a4f78bbd4d6dcffe587df'
t_authoringEndpoint = 'https://akchatbotde-test-lauth-ge-03.cognitiveservices.azure.com/'
t_predictionKey = 'e6bcc2a72adc4c22bfb1139e95959890'
t_predictionEndpoint = 'https://akchatbotde-test-lpred-ge-03.cognitiveservices.azure.com/'

 


# Stage Environment Credentials (2.1)
s_new_version="2.1"
s_app_id="4fdff778-8d0f-48bd-be01-85926d58bcc4"
s_authoringKey = 'dfac75a1fd1a4f78bbd4d6dcffe587df'
s_authoringEndpoint = 'https://akchatbotde-test-lauth-ge-03.cognitiveservices.azure.com/'
s_predictionKey = 'e6bcc2a72adc4c22bfb1139e95959890'
s_predictionEndpoint = 'https://akchatbotde-test-lpred-ge-03.cognitiveservices.azure.com/'
#   # The headers to use in this REST call.
l_headers = {
    "Ocp-Apim-Subscription-Key":"dfac75a1fd1a4f78bbd4d6dcffe587df"
}

 

# To Qna maker creds
to_q_kbid="b0db9cf6-6073-47e5-8613-ca200f5cccc6"
to_q_endpoint="https://akchatbotde-test-qcog-ge-03.cognitiveservices.azure.com/"
to_q_headers = {
    "Ocp-Apim-Subscription-Key":"e1889786583b4a53b5ff6a5f74051cbb"
}

 

# From QnA Maker Creds
from_q_kbid="03731eca-6f38-4ba3-bb50-8b8d99a23156"
from_q_endpoint="https://akchatbotde-test-qcog-ge-03.cognitiveservices.azure.com/"
from_q_headers = {
    "Ocp-Apim-Subscription-Key":"e1889786583b4a53b5ff6a5f74051cbb"
}

 


try:

 

    # Get Data from LUIS Test Envirnment
    response = requests.get(f'{t_authoringEndpoint}luis/api/v2.0/apps/{t_app_id}/versions/{t_current_version}/export', headers=l_headers)
    t_data=(response.json())
   
    #Push Data To LUIS Staging Environment
    requests.post(f'{t_authoringEndpoint}luis/api/v2.0/apps/{t_app_id}/versions/import?versionId={s_new_version}', headers=l_headers,data=response)
   
   
    # Train LUIS App
    client = LUISAuthoringClient(s_authoringEndpoint, CognitiveServicesCredentials(s_authoringKey))
    client.train.train_version(s_app_id, s_new_version)
    waiting = True
    while waiting:
        info = client.train.get_status(s_app_id, s_new_version)

 

        # get_status returns a list of training statuses, one for each model. Loop through them and make sure all are done.
        waiting = any(map(lambda x: 'Queued' == x.details.status or 'InProgress' == x.details.status, info))
        if waiting:
            print ("Waiting 10 seconds for training to complete...")
            time.sleep(10)
        else:
            print ("trained")
            waiting = False
           
    # Publish LUIS App
    responseEndpointInfo = client.apps.publish(s_app_id, s_new_version, is_staging=False)
   
    # Fetch Data from QnA Maker
    #requests.get(f'{t_authoringEndpoint}luis/api/v2.0/apps/{t_app_id}/versions/{t_current_version}/export', headers=headers)
   
   
    # Get Data from Test
    qna_response=requests.get(f'{from_q_endpoint}/qnamaker/v4.0/knowledgebases/{from_q_kbid}/test/qna',headers=from_q_headers)
   
    #qna_response=qna_response.json()
   
    #{Endpoint}/qnamaker/v4.0/knowledgebases/{kbId}
    requests.put(f'{to_q_endpoint}/qnamaker/v4.0/knowledgebases/{to_q_kbid}',headers=to_q_headers,data=qna_response)    
   
    # Publish KB
    #POST {Endpoint}/qnamaker/v4.0/knowledgebases/{kbId}
    requests.post(f'{to_q_endpoint}/qnamaker/v4.0/knowledgebases/{to_q_kbid}',headers=to_q_headers)    
    print(qna_response.status_code)

